In [1]:
import os
import pandas as pd
import sqlite3

from pdbbind_metadata_processor import PDBBindMetadataProcessor

In [2]:
dude_targets = os.listdir('../DUD-E/all/')

In [3]:
pdbbind_table = PDBBindMetadataProcessor().get_master_dataframe(remove_peptide_ligands=False)

In [4]:
pdbbind_table.head()

,PDB code,resolution,release year_x,-logKd/Ki,Kd/Ki,reference,ligand name,activity_list,sep,value,units,release year_y,Uniprot ID,protein name,active
0,3zzf,2.20,2012,0.40,Ki=400mM //,3zzf.pdf,(NLG),"[mM, =, 400]",=,400000000.0,nM,2012,Q01217,ACETYLGLUTAMATE KINASE,False
1,3gww,2.46,2009,0.45,IC50=355mM //,3gwu.pdf,(SFX),"[mM, =, 355]",=,355000000.0,nM,2009,O67854,TRANSPORTER,False
2,1w8l,1.80,2004,0.49,Ki=320mM //,1w8l.pdf,(1P3),"[mM, =, 320]",=,320000000.0,nM,2004,P62937,PEPTIDYL-PROLYL CIS-TRANS ISOMERASE A,False
3,3fqa,2.35,2009,0.49,IC50=320mM //,3fq7.pdf,(GAB&PMP),"[mM, =, 320]",=,320000000.0,nM,2009,P24630,"GLUTAMATE-1-SEMIALDEHYDE 2,1-AMINOMUTASE",False
4,1zsb,2.00,1996,0.60,Kd=250mM //,1zsb.pdf,(AZM),"[mM, =, 250]",=,250000000.0,nM,1996,P00918,CARBONIC ANHYDRASE 2,False


In [6]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../ChEMBL/chembl_29_sqlite/chembl_29.db")
df = pd.read_sql_query("""SELECT * 
FROM target_dictionary a 
JOIN target_components b ON a.tid = b.tid
JOIN component_sequences c ON b.component_id = c.component_id
JOIN component_class d ON c.component_id = d.component_id
JOIN protein_classification e ON d.protein_class_id = e.protein_class_id
JOIN component_synonyms f ON c.component_id = f.component_id""", con)

con.close()

In [11]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../ChEMBL/chembl_29_sqlite/chembl_29.db")
df = pd.read_sql_query("""SELECT accession, component_synonym, protein_class_desc 
FROM component_sequences c
JOIN component_class d ON c.component_id = d.component_id
JOIN protein_classification e ON d.protein_class_id = e.protein_class_id
JOIN component_synonyms f ON c.component_id = f.component_id
WHERE f.syn_type = 'GENE_SYMBOL'""", con)

con.close()

  accession component_synonym                  protein_class_desc
0    O09028             Gabrp            ion channel  lgic  gabaa
1    P02708            CHRNA1  ion channel  lgic  ach  chrn alpha
2    P04637              TP53                transcription factor
3    P04757            Chrna3  ion channel  lgic  ach  chrn alpha
4    P05106             ITGB3                   membrane receptor


In [16]:
pdbbind_table[pdbbind_table['Uniprot ID'].str.contains('--')].shape

(354, 15)

In [13]:
df['protein_class_desc'].value_counts()

unclassified                                        1271
enzyme  hydrolase                                    915
enzyme  transferase                                  865
enzyme  reductase                                    676
enzyme                                               675
                                                    ... 
enzyme  kinase  protein kinase  other  pek  gcn2       1
enzyme  cytochrome p450  cyp_2  cyp_2c                 1
enzyme  kinase  protein kinase  other  plk  plk2       1
enzyme  cytochrome p450  cyp_2  cyp_2b  cyp_2b1        1
enzyme  kinase  protein kinase  other  pek  pek        1
Name: protein_class_desc, Length: 670, dtype: int64

In [17]:
df['component_synonym'] = df['component_synonym'].str.lower()

In [23]:
pdbbind_chembl = pdbbind_table.merge(df, left_on='Uniprot ID', right_on='accession')

In [24]:
pdbbind_chembl

,PDB code,resolution,release year_x,-logKd/Ki,Kd/Ki,reference,ligand name,activity_list,sep,value,units,release year_y,Uniprot ID,protein name,active,accession,component_synonym,protein_class_desc
0,1w8l,1.80,2004,0.49,Ki=320mM //,1w8l.pdf,(1P3),"[mM, =, 320]",=,3.200000e+08,nM,2004,P62937,PEPTIDYL-PROLYL CIS-TRANS ISOMERASE A,False,P62937,ppia,enzyme isomerase
1,1w8m,1.65,2004,1.60,Ki=25mM //,1w8m.pdf,(E1P),"[mM, =, 25]",=,2.500000e+07,nM,2004,P62937,PEPTIDYL-PROLYL CIS-TRANS ISOMERASE A,False,P62937,ppia,enzyme isomerase
2,6gjy,1.29,2018,3.00,Kd>1000uM //,6gjy.pdf,(F1Z),"[uM, >, 1000]",>,1.000000e+06,nM,2018,P62937,PEPTIDYL-PROLYL CIS-TRANS ISOMERASE A,False,P62937,ppia,enzyme isomerase
3,6gjm,1.35,2018,3.58,Kd=266uM //,6gjm.pdf,(L36),"[uM, =, 266]",=,2.660000e+05,nM,2018,P62937,PEPTIDYL-PROLYL CIS-TRANS ISOMERASE A,False,P62937,ppia,enzyme isomerase
4,6gjj,1.38,2018,3.70,Kd=200uM //,6gjj.pdf,(L36),"[uM, =, 200]",=,2.000000e+05,nM,2018,P62937,PEPTIDYL-PROLYL CIS-TRANS ISOMERASE A,False,P62937,ppia,enzyme isomerase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13212,5lwe,2.80,2016,9.96,Kd=0.11nM //,5lwe.pdf,(79K),"[nM, =, 0.11]",=,1.100000e-01,nM,2016,P51686,C-C CHEMOKINE RECEPTOR TYPE 9,True,P51686,ccr9,membrane receptor 7tm1 peptide chemokine re...
13213,1g3m,1.70,2003 1,0.00,IC50=0.10nM //,1g3m.pdf,(PCQ),"[nM, =, 0.10]",=,1.000000e-01,nM,2003,P49888,ESTROGEN SULFOTRANSFERASE,True,P49888,sult1e1,enzyme transferase
13214,1n4k,2.20,2002 1,0.05,Kd=0.09nM //,1n4k.pdf,(I3P),"[nM, =, 0.09]",=,9.000000e-02,nM,2002,P11881,"INOSITOL 1,4,5-TRISPHOSPHATE RECEPTOR TYPE 1",True,P11881,itpr1,ion channel lgic ip3
13215,5zty,2.80,2019 1,0.12,Ki=0.075nM //,5zty.pdf,(9JU),"[nM, =, 0.075]",=,7.500000e-02,nM,2019,P34972,"G PROTEIN COUPLED RECEPTOR,T4 LYSOZYME,G PROTE...",True,P34972,cnr2,membrane receptor 7tm1 smallmol lipid-like ...


In [18]:
chembl_targets_in_dude = df[df['component_synonym'].isin(dude_targets)]

In [19]:
pdbbind_in_dude = pdbbind_table.merge(chembl_targets_in_dude, left_on='Uniprot ID', right_on='accession')

In [20]:
pdbbind_in_dude = pdbbind_table[pdbbind_table['Uniprot ID'].isin(chembl_targets_in_dude['accession'])]

In [21]:
pdbbind_in_dude[pdbbind_in_dude['protein name'] == 'BETA-SECRETASE 1']

,PDB code,resolution,release year_x,-logKd/Ki,Kd/Ki,reference,ligand name,activity_list,sep,value,units,release year_y,Uniprot ID,protein name,active
301,4zsm,1.96,2015,2.40,IC50~4mM //,4zsm.pdf,(4RW),"[mM, ~, 4]",~,4000000.000,nM,2015,P56817,BETA-SECRETASE 1,False
441,2ohk,2.20,2007,2.70,IC50~2000uM //,2ohk.pdf,(1SQ),"[uM, ~, 2000]",~,2000000.000,nM,2007,P56817,BETA-SECRETASE 1,False
444,3bra,2.30,2008,2.70,Kd=2000uM //,3bra.pdf,(AEF),"[uM, =, 2000]",=,2000000.000,nM,2008,P56817,BETA-SECRETASE 1,False
451,3hvg,2.26,2009,2.70,IC50=2mM //,3hvg.pdf,(EV0),"[mM, =, 2]",=,2000000.000,nM,2009,P56817,BETA-SECRETASE 1,False
552,3udh,1.70,2012,2.85,Kd=1.4mM //,3udh.pdf,(091),"[mM, =, 1.4]",=,1400000.000,nM,2012,P56817,BETA-SECRETASE 1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18743,1m4h,2.10,2002,9.52,Ki=0.3nM //,1m4h.pdf,(7-mer),"[nM, =, 0.3]",=,0.300,nM,2002,P56817,BETA-SECRETASE 1,True
18750,2g94,1.86,2006,9.52,Ki=0.3nM //,2g94.pdf,(ZPQ),"[nM, =, 0.3]",=,0.300,nM,2006,P56817,BETA-SECRETASE 1,True
18768,4wtu,1.85,2015,9.52,IC50=0.3nM //,4wtu.pdf,(3UT),"[nM, =, 0.3]",=,0.300,nM,2015,P56817,BETA-SECRETASE 1,True
19211,1xn2,1.90,2005 1,0.52,Ki=0.03nM //,1xn2.pdf,(11-mer),"[nM, =, 0.03]",=,0.030,nM,2005,P56817,BETA-SECRETASE 1,True


In [48]:
dict(pdbbind_in_dude['protein name'].value_counts())

{'BETA-SECRETASE 1': 343,
 'CELL DIVISION PROTEIN KINASE 2': 167,
 'TYROSINE-PROTEIN KINASE JAK2': 132,
 'GLYCOGEN PHOSPHORYLASE': 93,
 'CYCLIN-DEPENDENT KINASE 2': 88,
 'DIPEPTIDYL PEPTIDASE 4': 85,
 'EPIDERMAL GROWTH FACTOR RECEPTOR': 78,
 'BETA-LACTAMASE': 75,
 'GLUTAMATE RECEPTOR 2': 70,
 'PEROXISOME PROLIFERATOR-ACTIVATED RECEPTOR GAMMA': 67,
 'TYROSINE-PROTEIN KINASE SRC': 64,
 'ESTROGEN RECEPTOR': 61,
 'HEPATOCYTE GROWTH FACTOR RECEPTOR': 56,
 'SERINE/THREONINE-PROTEIN KINASE B-RAF': 39,
 'KINESIN-LIKE PROTEIN KIF11': 35,
 'SERINE/THREONINE-PROTEIN KINASE PLK1': 34,
 'TYROSINE-PROTEIN KINASE ABL1': 32,
 'CATECHOL O-METHYLTRANSFERASE': 31,
 'COLLAGENASE 3': 31,
 'POLY [ADP-RIBOSE] POLYMERASE 1': 29,
 'GLUTAMATE RECEPTOR, IONOTROPIC KAINATE 1': 26,
 "CGMP-SPECIFIC 3',5'-CYCLIC PHOSPHODIESTERASE": 26,
 'E3 UBIQUITIN-PROTEIN LIGASE XIAP': 24,
 'RETINOIC ACID RECEPTOR RXR-ALPHA': 23,
 'ANGIOTENSIN-CONVERTING ENZYME': 22,
 'FIBROBLAST GROWTH FACTOR RECEPTOR 1': 21,
 'HISTONE DEACETYLA